In [ ]:
#importing Libraries
import numpy as np
import glob,os
import pandas as pd
import tensorflow as tf
#from PIL import Image
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras import layers
from keras.layers import Dense,Flatten,Input
from keras.models import Model,Sequential
from tensorflow.keras.callbacks import ModelCheckpoint , EarlyStopping ,ReduceLROnPlateau
#from tensorflow.keras.applications.vgg19 import VGG19
#from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.efficientnet import EfficientNetB7
import warnings
warnings.filterwarnings("ignore")


In [ ]:
#Walk through the directory
train = '../input/animals10/raw-img'
translate = {"cane": "dog", "cavallo": "horse", "elefante": "elephant", "farfalla": "butterfly", "gallina": "chicken", "gatto": "cat", "mucca": "cow", "pecora": "sheep", "scoiattolo": "squirrel", "dog": "cane", "cavallo": "horse", "elefante" : "elephant", "farfalla": "butterfly", "gallina": "chicken", "gatto": "cat", "mucca": "cow", "ragno": "spider"}
print("Animals & directory info: ")
dataset = []
for i in os.listdir('../input/animals10/raw-img'):
    dataset.append(os.path.join(train , i))
for i in dataset:
    name = i.split('/')[-1]
    #print(name)
    print(name , "is translated as" , translate[name] , "and folder length is" , len(os.listdir(i)))
    

In [ ]:
image_array = []
image_label = []
for i in dataset:
    for n , j in enumerate(os.listdir(i)):
        if(n<1001):
            img_path = os.path.join(i , j)
            #print(img_path)
            e = cv2.resize(cv2.cvtColor(cv2.imread(img_path),cv2.COLOR_BGR2RGB),(224,224))
            image_array.append(e)
            l = i.split('/')[-1]
            image_label.append(translate[l])

In [ ]:
plt.imshow(image_array[1234])
plt.title(image_label[1234])
plt.show()

In [ ]:
# Converting labels into One Hot encoded sparse matrix
image_label = pd.get_dummies(image_label).values

# Converting train_images to array
image_array = np.array(image_array)


In [ ]:
X_train , X_test ,y_train ,y_test = train_test_split(image_array ,image_label ,test_size = 0.25 , random_state = 20)

In [ ]:
len(image_array)

In [ ]:
NUM_CLASSES = 10
IMG_SIZE = 224
size = [IMG_SIZE, IMG_SIZE]

en7 = tf.keras.applications.EfficientNetB7(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=size+[3],
    pooling=None,
    classes=10,
    classifier_activation="softmax",
    
)
for layer in en7.layers:
    layer.trainable = False
t7 = Flatten()(en7.output)

prediction7 = Dense(10, activation = 'softmax')(t7)
modelen7 = Model(inputs = en7.input , outputs = prediction7)
modelen7._name = 'model_effi_b7'
modelen7.summary()

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('model_effi_b7.h5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

modelen7.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"] )

histen7 = modelen7.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=15, verbose=1 , steps_per_epoch = 50 ,batch_size = 2, callbacks = [earlyStopping, mcp_save, reduce_lr_loss])

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()


plot_hist(histen7)

In [ ]:
nan